In [1]:
import numpy as np


In [2]:
# fitness function
def fitness_val(der_ale,A,zWT,O):
    z_i = zWT + np.matmul(A,der_ale)
    z_sq = np.square(z_i-O)
    w = np.ravel(np.exp(-np.sum(z_sq,axis=0))) #np.ravel to flatten matrix into a 1D array
    #log_w = np.log(w)
    return w

In [3]:
# hybridisation simulation with chance of choosing parent proportional to their fitness
def hybridisation_proportional_fitness(P1,P2,N,A,zWT,O,seedValue):
    rng = np.random.default_rng(seedValue)
    T = 1 #number of generations
    D = P1.shape[0]
    P1 = np.matrix(P1); P2 = np.matrix(P2)
    P1_fit = fitness_val(P1,A,zWT,O)
    P1_choice = np.random.choice(range(N),N,replace=True,p=(P1_fit)/sum(P1_fit))
    S1 = np.zeros([D,1])
    for i in P1_choice:
        s = P1[:,i]
        S1 = np.concatenate((S1,s),axis=1)
    S1 = np.delete(S1,0,1)
    P2_fit = fitness_val(P2,A,zWT,O)
    P2_choice = np.random.choice(range(N),N,replace=True,p=(P2_fit)/sum(P2_fit))
    # sample from the new population with replacement
    S2 = np.zeros([D,1])
    for i in P2_choice:
        s = P2[:,i]
        S2 = np.concatenate((S2,s),axis=1)
    S2 = np.delete(S2,0,1)

    np.place(S1,S1 == 1,rng.binomial(1,0.5,[D,N])*2)
    np.place(S2,S2 == 1,rng.binomial(1,0.5,[D,N])*2)
    F = np.matrix((S1 + S2)/2)

    while 1 in F:
        # sample two parental populations from the new population with replacement. Probability proportional to fitness
        F_fit_1 = fitness_val(F,A,zWT,O)
        P1_choice = np.random.choice(range(N),N,replace=True,p=(F_fit_1)/sum(F_fit_1))
        S1 = np.zeros([D,1])
        for i in P1_choice:
            s = F[:,i]
            S1 = np.concatenate((S1,s),axis=1)
        S1 = np.delete(S1,0,1)
        F_fit_2 = fitness_val(F,A,zWT,O)
        P2_choice = np.random.choice(range(N),N,replace=True,p=(F_fit_2)/sum(F_fit_2))
        # sample from the new population with replacement
        S2 = np.zeros([D,1])
        for i in P2_choice:
            s = F[:,i]
            S2 = np.concatenate((S2,s),axis=1)
        S2 = np.delete(S2,0,1)
        # a new hybridisation
        np.place(S1,S1 == 1,rng.binomial(1,0.5,[D,N])*2)
        np.place(S2,S2 == 1,rng.binomial(1,0.5,[D,N])*2)
        F = (S1 + S2)/2
        T += 1
        print(f"F {T}")
        print(F)
    H = np.mean(F)/2
    return F, T, H

# Random mating

This is a case where there is no selection (i.e. the A matrix is a zero matrix). Under such condition, the expected output should follow these patterns:

1. the final population should on average have a hybridisation index (H) of 0.5
2. time to fixation (T) should be proportional to N
3.